# ML Supervised-Decision trees and Random forests 


<!-- TOC START min:2 max:4 link:true asterisk:true update:true -->
* [What will you learn in this class?](#what-will-you-learn-in-this-class-)
* [Random Forest (regression and classification)](#random-forest-regression-and-classification)
  * [Building a random tree](#building-a-random-tree)
    * [General principle](#general-principle)
    * [Division criterion](#division-criterion)
      *[Eligibility](#eligibility)
    * [Stop criterion](#stop-criterion)
      *[Y quantitative](#y-quantitative)
      *[Y qualitative](#y-qualitative)
    * [Tree pruning](#tree-pruning)
    * [Construction of the nested suite of trees](#construction-of-the-nested-suite-of-trees)
  * [General remarks](#general-remarks)
<!-- TOC END -->



## What will you learn in this class?

This course will be very practical: the goal is to make you understand the theoretical principles of decision trees, which allow you to establish successive binary rules to classify observations or to make regressions. 

You will also learn the principle of bagging, which will be illustrated by switching from Decision Tree to Random Forest models.


## Decision Tree (regression and classification)

Decision Trees fall within the field of recursive partitioning methods, which are more commonly known by the acronym CART (_Classification And Regression Tree_). We speak of Regression when the target variable is quantitative, and of classification when the target variable is qualitative (categorical), a case that we will deal with in a second step. Here we are interested first in Regression Trees which are useful in the case of a quantitative target variable. Their advantage lies in the legibility of their graphical representation. The tree is composed of three types of elements:

* The root, where the set of learning data resides. At the root point, all training data is in one undifferentiated block.
* The nodes/branches, which represent, starting from the root, the points where the data is split into two subgroups according to a criterion related to the explanatory variables which goal is to achieve optimal prediction performance.
* Leaves, which are the terminal nodes of the tree and carry a prediction value when $Y$ is quantitative or a predicted class when $Y$ is qualitative.

Thus, starting from the root, a first node is defined which divides the data set according to a performance criterion linked to an explanatory variable and the target variable. For each of the two branches created, we repeat the same procedure, and so on. The tree is built sequentially, node by node, which results in smaller smaller subdivisions of the training data and as many possible predictions. A stopping criterion is used to stop creating new nodes and defining leaves also called terminal nodes.


![decision_tree](https://drive.google.com/uc?export=view&id=1TRyOAoMAkQ7kq1Hx3Zn7H66RqX2W0sGr)

This is an example of a random tree, with the root at the top, the first division branching out from the root, and the sequence of branches leading down to the leaves. The very clear and visual appearance of random trees is immediately noticeable and helps interpretation for reasonnably sized trees.

### Building a decision tree
#### General principle

The decision tree construction algorithm is structured as follows (in pseudo code):

* Initialization: The root is assigned all training data. The root is the first node.
* For each node, we attempt a division:
    * We select an explanatory variable $X$ and we define a cutoff criterion, a threshold if $X$ is quantitative, or a division in groups of modalities if $X$ is qualitative. In theory every devision criterion possible for every explanatory variable is tested, and the algorithm selects the one that minimises an error measure we will dive in later in the lecture. (Note that in scikit-learn only quantitative variables may be used, so the quantitative cutoff is used)
    * The node is marked as terminal if no admissible division can be found (we will explain this in details later in the lecture).
* The tree construction is interrupted when all nodes are terminal, and a predicted value or class is assigned to each terminal node, we will se how in what follows.

In order to complete this algorithm, we need several things:

1. A criterion for selecting the best division.
2. A set of rules to define whether a node is admissible and thus when to create terminal nodes.
3. A method for assigning a value or class to each terminal node or leaf.

#### Division criterion

### Admissibility

A node is **admissible** if the resulting branches have non-empty nodes (i.e. at least one observation belongs to each child node). For a parent node containing $M$ observations, there are $M - 1$ possible divisions if the variable selected for division is quantitative or qualitative ordinal, (and $2^{M-1}-1$ allowable divisions if the selected variable is nominal, however we mentionned that only quantitative variables may be used in scikit learn, all qualitative variables will have to be converted to numerical format).

In order to select the best admissible division, a **heterogeneity** function is defined, which has two properties :

* It equals to zero when all observation in a subset belong to the same modality of $Y$ when doing classification, or have the same value of $Y$ in case of a regression problem.
* It is maximal when the values of $Y$ are uniformely distributed in the node.

We thus seek the division which minimizes the sum of the heterogeneities of the child nodes. A different heterogeneity function will be defined for regression and classification models in what follows.

#### Stop criterion

A given node will be terminal in the following cases:
* when it is homogeneous, i.e. when all observations in the node have the same value or the same modality of $Y$
* when there are no more admissible divisions
* when the number of observations in the node is less than a predefined value, generally of the order of a few units. (in python this corresponds to the min_samples_split or min_samples_leaf arguments in scikit learn)
* when we reach a certain depth (a certain number of repeated subdivions)

### $Y$ quantitative

In the case of regression, the heterogeneity of node $k$ is written as follows :

## $H_k=\frac{1}{Card(k)}\cdot\sum_{i\in{k}}(y_i-\underline{y_k})^2$

$Card(k)$, sometimes also denoted $(k)$or $|k|$, is the number of elements in the $k$ node, and $\underline{y_k}$ is the mean of the values of $Y$ among the observations of the $k$ node, which is equivalent to the variance of node $k$. 
The division retained is the one for which $H_{kG}+H_{kD}$ : the sum of the heterogeneities of the left branch and the right branch is minimal.
Let's stop for a moment and think about this criterion. We are trying to find the divion which minimizes the sum of the variance of $Y$ in the right and left branch. The lower the variance of $Y$ in each branch, the more homogenous the observations in both branches are in terms of the target variable, if we build a model that is able to isolate groups of observations that have similar values of $Y$ in the same branch, then there is a high chance that it will be a good predictor!

### $Y$ qualitative

Let $Y$ be a qualitative variable with $m$ modalities or categories numbered from 1 to $m$. The heterogeneity function preferred most of the time is the GINI concentration:

### $H_k=\sum_{i=1}^{m}p_{k}^{i}\cdot(1-p_{k}^{i})$

Where $p_{k}^{i}$ is the proportion of the class $i$ of $Y$ in the $k$ node. Let's now analyse this formula, what happens when all observations in a branch belong to the same modality $y_j$ of $Y$, then $p_{i}=0 \forall i \neq j$ and $1 - p_{j} = 0$ which means the heterogeneity is 0. This formula reaches its maximum value when values of $Y$ are uniformely distributed in the branch. Again we are trying to minimize the sum of the heterogeneity of the branches, meaning the deeper we go into the tree, the more homogeneous the branches become, meaning more and more observations in the node belong to the same class of $Y$.

#### Fight overfitting

We saw earlier with the LASSO and RIDGE models that a significant risk in any supervised learning problem is overfitting. The stopping criterion defined for tree construction is critical in limiting overfitting. With a very loose stopping criterion, the tree will be able to grow to its fullest and terminal nodes will contain very few observations for which $Y$ will most probably be perfectly homogeneous. Such a decision tree will most likely have a near perfect score on the training data, because it has been able to fit every single observation in there, however, this only works when observations in the test set are identical to training examples, any slight difference in variable distribution in the test set will yield very unpredictable results, potentially good, but most likely incorrect. That is because we let the bias reduce to zero and the variance blow through the roof, which makes for a model that will most likely generalize poorly when applied to new data.

The goal is to find an intermediate tree to achieve a preferable bias/variance trade-off for the needs of the estimation of $Y$. The number of sub-trees is often unfortunately very high and it would take a long time to check every possible subtree's performance.

Instead, what you could do is play with stopping criteria using gridsearch to find the model's hyperparameters that would maximize the prediction performance on the test set.

Reminder : the stopping criteria you can play with to achieve optimal bias / variance compromise are
* min_samples_leaf
* min_samples_split
* max_depth

### General remarks

* The algorithm tends to pay less attention to qualitative variables if they are converted into dummy variables  That is because the explanatory power of the variable is split over as many variables as they are modalities. A general strategy would be to gather modalities that are less common or modalities that represent observations with similar distribution of $Y$.

* Decision trees do not require special assumptions about the distributions of the variables (no normalization needed).

* Since a single variable is selected to create each node, the model is able to perform implicit variable selection. However, since only one variable is used for each split, less informative variables might not be included in the model and only a portion of the total inforlation in the data will be used.

* The search for a division depends solely on the relative values of the values of the quantitative explanatory variables, so the algorithm is resistant to atypical values and asymmetric value distributions.

* The hierarchical structure of the tree (divisions are made one by one) favors the propagation of the error generated by a division to all child nodes. The hierarchical structure of the tree (divisions are done one by one) favors the propagation of the error generated by a division to all child nodes. The decision trees can thus miss a global optimum and thus the true classification or regression function that links the explanatory variables to the target variable.

* In the case of regression, the result of the tree is a stepped function, all observations on the same leaf will take the same estimated value of $Y$.

* If the true function has regularity or smoothness properties (for example a polynomial or an affine line), these properties will not be retained by the random tree model.
